# Lie Detection Based on Facial Expression and Body Posture

> Mayur Sharma<br>
> Khushi Tulsian<br>
> Rohan deep Kujur<br>

In [28]:
import mediapipe as mp
import cv2
from math import inf
import pandas as pd
import numpy as np

mp_holistic = mp.solutions.holistic #Sub-module
holistic    = mp_holistic.Holistic() #Class
mp_drawing  = mp.solutions.drawing_utils #Visualizing the data points

import warnings
warnings.filterwarnings('ignore')

In [29]:
# eyebrow                     x2
# face left right top bottom  x4
# lips                        x1
# body                        x8
# =15 columns     x and y     = 30 features

columns0 = ['lips', 'left_brow', 'right_brow'] + [f"body{i}" for i in range(8)] + [f"face{i}" for i in range(4)] #Column Names
columns  = list()
for col in columns0:
    columns.append(col+'x')
    columns.append(col+'y')
columns.append('TRUTH')# Column for target value
#Empty Dataframe with same column names
TRAINING_FEATURES = pd.DataFrame(columns=columns) 
TESTING_FEATURES  = pd.DataFrame(columns=columns)

LANDMARKS_LOC = {
    'left_brow'  : [107, 66, 105, 63, 70, 46,53,52,65,55],
    'right_brow' : [336, 285, 296, 295, 334, 282, 293, 283, 276, 300],
    'lips'       : [78, 308, 80, 88, 82, 87, 312, 317, 310, 318],
    'face0'      : [54,68,103,104,108,69,67,10,151,338,337,397,333,332,298,284,251,301,21,71,109,297,299],                                                                       #forehead
    'face1'      : [18, 32, 83, 140, 148, 152, 171, 175, 176, 199, 200, 201, 208, 262, 313, 369, 377, 396, 400, 421, 428],                                                       #chin
    'face2'      : [36, 50, 58, 93, 101, 111, 116, 117, 118, 123, 132, 137, 138, 147, 172, 177, 186, 187, 192, 203, 205, 206, 207, 212, 213, 214, 215, 216, 227, 228, 234],      #left_face
    'face3'      : [266, 280, 288, 323, 330, 340, 345, 346, 347, 352, 361, 366, 367, 376, 397, 401, 410, 411, 416, 423, 425, 426, 427, 432, 433, 434, 435, 436, 447, 448, 454]   #right_face
}

#video intervals where the subject answers
INTERVALS = [
    [90,  140, 1],
    [170, 220, 1],
    [250, 340, 0],
    [370, 450, 1],
    [480, 540, 1],    #TESTING INTERVAL
    [600, 660, 0],
    [inf, inf, 0]]

TEST_INTERVAL = 3

TRAINING_FEATURES

,lipsx,lipsy,left_browx,left_browy,right_browx,right_browy,body0x,body0y,body1x,body1y,...,body7y,face0x,face0y,face1x,face1y,face2x,face2y,face3x,face3y,TRUTH


## Drawing and marking data points, extracting and normalizing them

In [30]:
def draw_holistics(image, results): #Draw the data points on the image/video
    mp_drawing.draw_landmarks( frame,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        mp_drawing.DrawingSpec(
            color=(255,0,255),
            thickness=1,
            circle_radius=1),
        
        mp_drawing.DrawingSpec(
            color=(255,255,255),
            thickness=1,
            circle_radius=1)
        )
    
    mp_drawing.draw_landmarks(
        frame,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(
            color=(0,230,255),
            thickness=2,
            circle_radius=1),
        
        mp_drawing.DrawingSpec(
            color=(255,255,255),
            thickness=1,
            circle_radius=1)
        )

def extract_facial_features(landmarks): #Facial feature extraction and normalization
    nose = landmarks[4]  #nose at index 4
    #x, y = get_cordinates(nose, DIMS)
    #cv2.circle(frame, (x, y), 2, (0,0,0), 1)
    features = dict()
    
    for feature in LANDMARKS_LOC:
        feature_loc = np.array([0,0], dtype=np.float64)#array to store feature co-ordinates(x&y)
    
        for idx in LANDMARKS_LOC[feature]:
            mark = landmarks[idx]
            feature_loc += np.array( [mark.x, mark.y] )
        feature_loc /= len(LANDMARKS_LOC[feature])           #average feature location
        
        result = feature_loc - np.array([nose.x, nose.y])    #normailze
        features[feature+'x'] = result[0]
        features[feature+'y'] = result[1]
    
    return features #Returning a dicionary of facial {features: co-ordiante}

def extract_body_features(landmarks):
    #(Left shoulder + Right shoulder) /2 = chest
    chest = [(landmarks[11].x + landmarks[12].x)/2, (landmarks[11].y + landmarks[12].y)/2]
    i = 0
    features = dict()
    
    for idx in range(11,19):
        mark = landmarks[idx]
        features[f'body{i}x'] = mark.x - chest[0] #body0,Body1,body2,body3
        features[f'body{i}y'] = mark.y - chest[1]
        i += 1
    
    return features

In [38]:
video = cv2.VideoCapture('.\\Data\\Train_video.mp4')
DIMS  = np.array([500, 880], dtype=int)      #Video crop dimension to subject
SCALE = 0.75

TRAINING_FEATURES.drop(TRAINING_FEATURES.index, inplace=True)
TESTING_FEATURES.drop(TESTING_FEATURES.index, inplace=True)

frame_no = 0
curr_interval  = 0

while(1):
    #reading video data
    success, frame = video.read()
    if not success: break
    if frame_no == 400: break     #Testing
    
    #resizing and cropping
    frame = cv2.flip(frame,0)
    frame =cv2.flip(frame,1)
    frame = frame[300:300+DIMS[1], 150:150+DIMS[0]]
    frame = cv2.resize(frame, (0, 0), fx = SCALE, fy = SCALE)

    if INTERVALS[curr_interval][0] <= frame_no <= INTERVALS[curr_interval][1]:
        #processing body features
        results = holistic.process( cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) )
        draw_holistics(frame, results)

        face_features = extract_facial_features(results.face_landmarks.landmark)
        body_features = extract_body_features(results.pose_landmarks.landmark)
        #print(face_features)
        #print(len(body_features) + len(face_features))

        features = {"TRUTH" : INTERVALS[curr_interval][2]}
        features.update(face_features)
        features.update(body_features)
        
        #Testing interval for our code
        if curr_interval != TEST_INTERVAL:
            TRAINING_FEATURES = TRAINING_FEATURES.append([features])
        else:
            TESTING_FEATURES = TESTING_FEATURES.append([features])
        
        if frame_no > INTERVALS[curr_interval][1]-1:
            curr_interval += 1
    
    #display results
    
    cv2.imshow("Frame", frame)
    cv2.waitKey(1)
    frame_no += 1

cv2.destroyAllWindows()
video.release()

In [39]:
TRAINING_FEATURES["TRUTH"] = pd.to_numeric(TRAINING_FEATURES["TRUTH"])
TRAINING_FEATURES

,lipsx,lipsy,left_browx,left_browy,right_browx,right_browy,body0x,body0y,body1x,body1y,...,body7y,face0x,face0y,face1x,face1y,face2x,face2y,face3x,face3y,TRUTH
0,-0.002419,0.030762,-0.054171,-0.032112,0.057202,-0.028707,0.251291,-0.004178,-0.251291,0.004178,...,0.546542,0.005830,-0.037163,-0.005098,0.059953,-0.080336,0.018579,0.075834,0.022655,1
0,-0.002251,0.030593,-0.053275,-0.032740,0.058877,-0.028978,0.251189,-0.004239,-0.251189,0.004239,...,0.547065,0.007365,-0.037581,-0.005615,0.059516,-0.080003,0.018589,0.075953,0.022806,1
0,-0.002404,0.030721,-0.053282,-0.032610,0.058597,-0.028616,0.251088,-0.004261,-0.251088,0.004261,...,0.547675,0.007215,-0.037382,-0.006096,0.059705,-0.080528,0.018391,0.075447,0.022971,1
0,-0.003146,0.030959,-0.052902,-0.032612,0.058993,-0.028304,0.251161,-0.004133,-0.251161,0.004133,...,0.547993,0.007709,-0.037250,-0.007137,0.059861,-0.080898,0.018418,0.075046,0.023525,1
0,-0.002866,0.030874,-0.052397,-0.032378,0.058962,-0.028206,0.251406,-0.004032,-0.251406,0.004032,...,0.548212,0.007908,-0.037060,-0.006740,0.059886,-0.080275,0.018453,0.075268,0.023538,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-0.002263,0.026674,-0.054149,-0.040650,0.060375,-0.035338,0.252967,-0.005187,-0.252967,0.005187,...,0.545576,0.007581,-0.047217,-0.006334,0.054058,-0.080865,0.007568,0.074918,0.013289,0
0,-0.002713,0.027034,-0.053923,-0.040567,0.060210,-0.035064,0.252933,-0.005209,-0.252933,0.005209,...,0.545678,0.007655,-0.047118,-0.006928,0.054331,-0.081279,0.007771,0.074458,0.013761,0
0,-0.002980,0.026931,-0.053768,-0.040825,0.060496,-0.034996,0.252888,-0.005141,-0.252888,0.005141,...,0.545560,0.007870,-0.047244,-0.007448,0.054503,-0.081763,0.007570,0.074220,0.013964,0
0,-0.003624,0.027335,-0.053449,-0.040566,0.060733,-0.034779,0.252832,-0.005162,-0.252832,0.005162,...,0.545581,0.008121,-0.046856,-0.008493,0.055233,-0.082203,0.008406,0.073773,0.014958,0


In [40]:
from sklearn.linear_model import LogisticRegression

TRAINING_Y = TRAINING_FEATURES["TRUTH"]
TRAINING_X = TRAINING_FEATURES.drop("TRUTH", axis=1)

TESTING_Y = TESTING_FEATURES["TRUTH"]
TESTING_X = TESTING_FEATURES.drop("TRUTH", axis=1)

logr = LogisticRegression(random_state=16)
logr = logr.fit(TRAINING_X, TRAINING_Y)

In [41]:
y_pred = logr.predict(TESTING_X)
print(f"Expected output: {INTERVALS[TEST_INTERVAL][2]}")
print("Prediction:\n", y_pred)

Expected output: 1
Prediction:
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


## Conclusion

The predicted values match the actual values, hence our model works successfully!

### Extra Notes
1) FACE MESH: **landmarks index 4** is the tip of the nose used for normalisation